In [4]:
import pandas as pd
%matplotlib inline

import boto3
from io import StringIO

In [5]:
# df=pd.read_csv("ml_log.csv",parse_dates=True)#infer_datetime_format=True) ,index_col=['time','ticker']

In [6]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="model-support-files"
Key="4_ml_log.csv"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df = pd.read_csv(read_file['Body'],sep=',',index_col=['datetime'],parse_dates=True)

In [7]:
df=df.loc[:, df.columns != 'ATR']
df.head()

,security,close,RSI,STOCHASTIC
datetime,,,,
2010-12-30 22:00:00,EUR_USD,1.32855,NaN,NaN
2010-12-31 22:00:00,EUR_USD,1.33815,NaN,NaN
2011-01-01 22:00:00,EUR_USD,1.33280,NaN,NaN
2011-01-02 22:00:00,EUR_USD,1.32512,NaN,NaN
2011-01-03 22:00:00,EUR_USD,1.32919,NaN,NaN


In [8]:
# df['datetime']=pd.to_datetime(df['datetime'])
df['fwd_returns']=df.groupby("security")["close"].pct_change(5)

In [9]:
df.sort_values(by='datetime',inplace=True)

In [10]:
df=df.reset_index().drop(columns=['datetime','security','close'])

In [11]:
df.head(100)

,RSI,STOCHASTIC,fwd_returns
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
95,64.126022,51.751124,0.010203
96,57.309926,28.097778,0.019659
97,66.327621,72.346196,0.017007
98,66.191288,84.836598,0.015130


In [12]:
df.dropna()

,RSI,STOCHASTIC,fwd_returns
58,56.991619,78.882076,-0.005146
60,59.448606,77.279931,0.003864
62,60.945457,79.322084,0.013553
63,67.632318,67.385341,0.019120
64,56.546196,72.484186,-0.000088
...,...,...,...
4835,41.858895,-12.817442,-0.008176
4836,41.948026,-11.469939,-0.008326
4837,33.961128,-26.089538,-0.019533
4838,43.654748,-5.028193,-0.001247


In [9]:
# df.dropna().to_csv('ml_log_processed.csv',index=False)

In [10]:
df.dropna().to_csv(("processed_"+Key),index=False)

In [12]:
csv_buffer = StringIO()
df.to_csv(csv_buffer,index=False)
s3.put_object(Bucket=Bucket, Key=("processed_"+Key),Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '15E4E729C85BFEB8',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"e45b7db678a861fe584969abaf0a0658"',
   'server': 'MinIO/RELEASE.2019-12-24T23-04-45Z',
   'vary': 'Origin',
   'x-amz-request-id': '15E4E729C85BFEB8',
   'x-xss-protection': '1; mode=block',
   'date': 'Sun, 29 Dec 2019 17:18:56 GMT'},
  'RetryAttempts': 0},
 'ETag': '"e45b7db678a861fe584969abaf0a0658"'}

### Old Format

In [2]:
# df=pd.read_csv("ml_log.csv",index_col=['time','ticker'],parse_dates=True)
df=pd.read_csv("ml_log.csv",index_col=['time'],parse_dates=True)

In [3]:
df.head()

,ticker,close,sma1,sma2
time,,,,
2019-01-28 22:00:00,EUR_USD,1.14113,1.135365,1.137738
2019-01-28 22:00:00,GBP_USD,1.30556,1.294032,1.280558
2019-01-29 22:00:00,EUR_USD,1.14063,1.135649,1.138145
2019-01-29 22:00:00,GBP_USD,1.30543,1.296327,1.282924
2019-01-30 22:00:00,EUR_USD,1.14360,1.136307,1.138780


In [16]:
df['fwd_returns']=df.groupby("ticker")["close"].pct_change(1)

In [17]:
df=df.reset_index().drop(columns=['time','ticker','close'])

In [18]:
df.dropna().to_csv("ml_log_processed.csv",index=False)

In [19]:
df.head()

,sma1,sma2,fwd_returns
0,1.135365,1.137738,NaN
1,1.294032,1.280558,NaN
2,1.135649,1.138145,-0.000438
3,1.296327,1.282924,-0.000100
4,1.136307,1.138780,0.002604


## QGRID

In [51]:
# ! pip install qgrid

# ! jupyter nbextension enable --py --sys-prefix qgrid
# ! jupyter nbextension enable --py --sys-prefix widgetsnbextension

# import qgrid
# qgrid_widget=qgrid.show_grid(df,show_toolbar=True)
# qgrid_widget